* Collect data from the internet with automatic downloading. <br>
* Activate the free windscribe VPN. For this purpose download the VPN on your system (per ubuntu command shell: https://www.geeksforgeeks.org/how-to-setup-vpn-on-ubuntu-linux-system-for-ip-spoofing-using-windscribe/, thorugh official webpage: https://windscribe.net/download ). Through the VPN we can change the IP, when the daily download limit of one webpage is already reached. However, the free version of windscribe has of course also its montly usage limit. To solve this, you can create several windscribe accounts.<br>
Without delivering an e-mail address in account creation two GB are freely available per month. <br>
You can make use of the following account: <br>
User name: stage1_BA_MIDI <br>
Password: findMIDI
* As web driver and url opening browser Firefox is used. Therefore, you need Firefox as a webbrowser. Sometimes (around 50 downloads later) the driver dies. Reboot the computer to make it work again. When running the notebook answer 'Please, type starting index in it:' with the index of the sample to start.<br>
* The webpage 'lvbeethoven.fr' is opened by url automatically instead of downloading because its structure is mixed up. Therefore, its the job of the user to download the belonging midi file. <br>
* Finally, download manually missing classic titles from kunstderfuge.com. All missing classic titles should be listed in the file 'final_missing_titles'. Delete manually found titles from there.

In [1]:
import numpy as np

import pandas as pd

import random
import os # also needed to start the VPN 

import re

# open webpages:
import subprocess as sp
import webbrowser

# work with content on webpages:
# approach 1:
import requests 
from bs4 import BeautifulSoup
# approach 2: 
# Has a function to click on objects like we would do with the mouse (.click())
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait # wait until a webpage counter goes to 0 or has loaded
from selenium.webdriver.common.action_chains import ActionChains # go to a certain element of a webpage


# for virus detection:
from virustotal_python import Virustotal, VirustotalError 

import pprint # from pprint import pprint

# for renaming file on which most lately worked
import shutil

import time

import magic # to see the real file type without its extension, like '.txt'

In [2]:
import sys
sys.path.insert(0, '../../stage-1/overall_used_tools')
import requirements_check as rc

import bs4, selenium, virustotal_python
rc.check(sys, [np,pd,random,os,re,sp,webbrowser,requests,bs4,selenium,virustotal_python,pprint,shutil,time,magic])

Your version identical with original version 
-> numpy 
-> 1.21.6


Your version identical with original version 
-> pandas 
-> 1.3.5


For 
random 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


For 
os 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Your version identical with original version 
-> re 
-> 2.2.1


For 
subprocess 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


For 
webbrowser 
python version needs to fit 
-> original version 3.7.3 
-> your version 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


Your version identical with original version 
-> requests 
-> 2.28.1


Your version identical with original version 
-> bs4 
-> 4.9.1


Your version identical with original version 
-> selenium 
-> 4.1.3


Possibly different versions: 
-> v

## Getting information out of dataset

In [3]:
path = '../../stage0_provided_information/gems-emotion-tags-main/data/GEMS_songs_overview.csv'
small = pd.read_csv(path, skiprows=1)
small2 = small[['Code','MP3-Code','Artist','Title']]
small3 = small2.dropna(how='all') # remove all rows only containing nan
small3

,Code,MP3-Code,Artist,Title
0,H001,H_Trailerpark_Schlech,Trailerpark,Schlechter Tag
1,H002,H_Desiigner_Panda,Desiigner,Panda
2,H003,H_NAS_DooRags,NAS,Doo Rags
3,H004,H_DMX_XGonGiv,DMX,X Gon' Give It To Ya
4,H005,H_Xatar_MeineGr,Xatar,Meine Große Liebe
...,...,...,...,...
365,P142,P_Oasis_DontLoo,Oasis,Don't Look Back In Anger
366,P143,P_DerJungemitderGitarre_HalloWo,Der Junge mit der Gitarre,Hallo Worum Gehts Ich Bin Dagegen
367,P144,P_ChrisBrown_Forever,Chris Brown,Forever
368,P145,P_RyanAdams_LuckyNo,NaN,Ryan Adams


In [4]:
number_term_nan=0
number_mp3_nan=0
col = small3.columns
for i ,row in small3.iterrows():
    curr_code = row[col[0]]
    curr_mp3 =  row[col[1]]
    curr_art =  row[col[2]]
    curr_song =  row[col[3]]
    
    
    if pd.isna(curr_art) or pd.isna(curr_song):
        number_term_nan +=1
        print(row)
    
    if pd.isna(curr_mp3): # replace missing MP3-ID with code 
        small3.at[i, 'MP3-Code'] = small3.at[i, 'Code']
        number_mp3_nan+=1


print('number of nan-s for title and/or artist:', number_term_nan)
print('number nan-s for mp3 code: ', number_mp3_nan)

Code                       P145
MP3-Code    P_RyanAdams_LuckyNo
Artist                      NaN
Title                Ryan Adams
Name: 368, dtype: object
Code                          P146
MP3-Code    P_AmyWinehouse_BackToB
Artist                         NaN
Title                Amy Winehouse
Name: 369, dtype: object
number of nan-s for title and/or artist: 2
number nan-s for mp3 code:  3


In [5]:
# The mixed up lines from above are corrected:
small3.at[368, 'Artist'] = 'Ryan Adams'
small3.at[368, 'Title'] = 'Lucky Now'

small3.at[369, 'Artist'] = 'Amy Winehouse'
small3.at[369, 'Title'] = 'Back to Back'

In [6]:
# Correct falsly written titles and artists:
small3.at[254, 'Artist'] = 'Katy Perry' # was orignally written 'Katie Perry' what is wrong; correct it here
small3.at[238, 'Title'] = 'Poker Face' # original: 'Pokerface'

In [7]:
# create folders in which data shall be stored
codes = small3['MP3-Code']

path = 'midi-files'
os.makedirs(path, exist_ok=True)

for i in codes:
    path2 = os.path.join(path, str(i))
    try:
        os.mkdir(path2)
    except:
        pass
    
    try: 
        path3 = os.path.join(path2, 'demo')
        path3a = os.path.join(path3, 'webpageinfo.txt')
        
        path4 = os.path.join(path2, 'full')
        path4a = os.path.join(path4, 'webpageinfo.txt')
        
        
        os.mkdir(path3)
        open(path3a,'w') # create file in which the related web pages shall be stored
        
        os.mkdir(path4)
        open(path4a,'w')
        
    except:
        pass

In [8]:
# Add columns to our dataframe, these will be used to search for the certain samples in the web:
small3['search_for'] = small3['Artist'] + ' '+ small3['Title']
small3['search_for2'] = small3['Artist'] + ' - '+ small3['Title']
small3

/home/c/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/c/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Code,MP3-Code,Artist,Title,search_for,search_for2
0,H001,H_Trailerpark_Schlech,Trailerpark,Schlechter Tag,Trailerpark Schlechter Tag,Trailerpark - Schlechter Tag
1,H002,H_Desiigner_Panda,Desiigner,Panda,Desiigner Panda,Desiigner - Panda
2,H003,H_NAS_DooRags,NAS,Doo Rags,NAS Doo Rags,NAS - Doo Rags
3,H004,H_DMX_XGonGiv,DMX,X Gon' Give It To Ya,DMX X Gon' Give It To Ya,DMX - X Gon' Give It To Ya
4,H005,H_Xatar_MeineGr,Xatar,Meine Große Liebe,Xatar Meine Große Liebe,Xatar - Meine Große Liebe
...,...,...,...,...,...,...
365,P142,P_Oasis_DontLoo,Oasis,Don't Look Back In Anger,Oasis Don't Look Back In Anger,Oasis - Don't Look Back In Anger
366,P143,P_DerJungemitderGitarre_HalloWo,Der Junge mit der Gitarre,Hallo Worum Gehts Ich Bin Dagegen,Der Junge mit der Gitarre Hallo Worum Gehts Ic...,Der Junge mit der Gitarre - Hallo Worum Gehts ...
367,P144,P_ChrisBrown_Forever,Chris Brown,Forever,Chris Brown Forever,Chris Brown - Forever
368,P145,P_RyanAdams_LuckyNo,Ryan Adams,Lucky Now,Ryan Adams Lucky Now,Ryan Adams - Lucky Now


## Helper functions for downloading

In [38]:
def answer():
    '''When webpage response != 200, activate this function'''
    
    #answer = input('Webpage could not be opened. Shall be continued?[y/n]')
    #while answer != 'y':
    #    answer = input('Webpage could not be opened. Shall be continued?[y/n]')
    return 

In [10]:
def open_(url):
    '''function to open url for manual download'''
    
    print('Please, have a look at the opening webpage (Firefox used) and download the wished midi file(s)...')
    print('WARNING: The current webpage will be opened on Firefox. THIS jupyter notebook shall be run on an other browser because Firefox will be closed automatically!')
    print('In this case you have to type in \n{search_term2}\n in the search bar of the webpage to find the file.')
    #webbrowser.open(url) # open webpage for the use

    child = sp.Popen("firefox %s" % url, shell=True)

    answer = input('Are you done with looking?[y/n]')
    while answer != 'y':
        answer = input('Are you done with looking?[y/n]')
    os.system("pkill "+"firefox")

    return

In [11]:
def download(download_url:str, download_path_mp3code:str, download_page:str):
    '''download a file by only using the url.'''
    
    # check which name the file to download will get. We count the number of files in the folder such that
    # they won't get the same name: e. g., file1, file2, file3 as resulting files
    dir_name = os.path.dirname(download_path_mp3code)
    number_files = len(os.listdir(dir_name))
    
   # download_url = 'http://www.rppmf.com/coldplay/2000-parachutes/05-coldplay_2000-yellow-[k].mid'
    
    # download from certain webpage:
    r = requests.get(download_url)
    
    download_path_mp3code_splitted = download_path_mp3code.split('.')
    mid_path_name = download_path_mp3code_splitted[0] + '_' + str(number_files) + '.mid'# + download_path_mp3code_splitted[1] 
    with open(mid_path_name,'wb') as f: 
        f.write(r.content)
    
    
    # write in txt file in folder from which webpage we downloaded midi file:
   # textfile_path = dir_name + '/webpageinfo.txt'
    #with open(textfile_path, 'a') as t:
     #   t.write(download_page)
      #  t.write('\n')
        
    full_path_split = os.path.realpath(dir_name).split('.')
    full_path_name = full_path_split[0] + '_' + str(number_files) + '.' + download_path_mp3code_splitted[1]
    return mid_path_name #, full_path_name

In [12]:
def rename_downloaded_file(download_path_mp3code:str, download_page:str):
    '''change name of the file downloaded most lately'''
    
    # check which name the file to download will get. We count the number of files in the folder such that
    # they won't get the same name: e. g., file1, file2, file3 as resulting files
    
    dir_name = os.path.dirname(download_path_mp3code)
    number_files = len(os.listdir(dir_name))
    
    
    # create wished file name:
    file_name = os.path.basename(download_path_mp3code)
    file_name_split = file_name.split('.')
    storage_name = file_name_split[0] + '_' + str(number_files) + '.mid'# + file_name_split[1]
    
    # find storage path:
    full_path = os.path.abspath(download_path_mp3code)
    full_path = full_path.replace(file_name, '')
    print(full_path)
    # Give file last time touched the new name:
    filename_ = max([full_path  + f for f in os.listdir(full_path)],key=os.path.getctime)
    shutil.move(filename_,os.path.join(full_path,storage_name))
    #time.sleep(10)

    

    # get absolute path to return:
    download_path_mp3code_splitted = download_path_mp3code.split('.')
    mid_path_name = download_path_mp3code_splitted[0] + '_' + str(number_files) + '.' + download_path_mp3code_splitted[1] 
    
    return mid_path_name#os.path.join(full_path,storage_name)


In [13]:
def webpage_virus_check(url:str):
    ''' URL virus check '''
    
    global day_limit_counter
    day_limit_counter += 1
    return 1
    # Retrieved from: 
    # https://pypi.org/project/virustotal-python/
    vtotal = Virustotal(API_KEY='7d3d966b2b57f558b1d6097b8ee5a5e18dd92c68a1eedd03e02caef10db22ed2')
    #vtotal = Virustotal(API_KEY='60b7611fa8b76bc01429c6ef111656f1088c65fe565a5e011fa91c94feb0acba')
    # these are virus total ID keys. We have a daily checking limit of 500 files/urls. When the first
    # key does not work anymore, use the second one. When both reached their limits create a new key on 
    # https://pypi.org/project/virustotal-python/ or wait a day. 
    
    done = False
    while done == False:
        try:
            resp = vtotal.request("url/scan", params={"url": url}, method="POST")
     
            # Send a URL to VirusTotal for analysis
            url_resp = resp.json()
            # Obtain scan_id
            scan_id = url_resp["scan_id"]
            # Request report for URL analysis
            analysis_resp = vtotal.request("url/report", params={"resource": scan_id})

            malware_suspciton = len(re.findall('malware site', str(analysis_resp.json())))
            malicious_suspciton = len(re.findall('malicious site', str(analysis_resp.json())))
            
  
            if (malware_suspciton + malicious_suspciton) > 2:

                
                return -1
        
            elif (malware_suspciton + malicious_suspciton) > 0 or ('Resource does not exist in the dataset' == analysis_resp.json()['verbose_msg']):

                return 0
            
            else: # clean url
                return 1
                
            done = True

        except: # because it is no premium API key, we can only make 4 virus checks per minute.
            print('wait url check....')
            time.sleep(60)

    '''
    free API key limits:

    Request rate	4 lookups / min
    # thats problematically slow
    # set time.sleep(60) after 4 lookups
    # you could look at each webpage once for the virus scan with assumption either all files with virus or all
    # files clean.


    Daily quota	500 lookups / day
    Monthly quota	15.50 K lookups / month
    ''';



In [14]:
def file_virus_check(FILE_PATH:str):
        ''' file virus check, FILE_PATH e. g. 'yo/err.MID'  '''
        
        global day_limit_counter
        day_limit_counter += 1
        
        return 1

        done = False
        vtotal = Virustotal(API_KEY='7d3d966b2b57f558b1d6097b8ee5a5e18dd92c68a1eedd03e02caef10db22ed2')
        #vtotal = Virustotal(API_KEY='60b7611fa8b76bc01429c6ef111656f1088c65fe565a5e011fa91c94feb0acba')
        while done == False:
            try:
                # Create report:

                # Create dictionary containing the file to send for multipart encoding upload
                files = {"file": (os.path.basename(FILE_PATH), open(os.path.abspath(FILE_PATH), "rb"))}
                
                # v2 example
                resp = vtotal.request("file/scan", files=files, method="POST")

                # The v2 API returns a response_code
                # This property retrieves it from the JSON response

                # Print JSON response from the API:

                # Call report content:
                FILE_ID = str(resp.json()['scan_id']) # get the ID for of the file scan process

                resp = vtotal.request("file/report", {"resource": FILE_ID})
              
                if "\'detected\': True" in str(resp.data):
                    os.remove(os.path.abspath(FILE_PATH))
                    print('file dangerous removed')
                    return -1

                else: # file seems to be okay
                    return 1
                
                done = True
                
            except:
                print('wait file check....')
                time.sleep(60)


In [15]:
def scroll_shim(passed_in_driver, object):
    ''' needed when we want to find an object by location. The reason for this is when we get an error 
    messeage that one element overlays the other one.'''
    
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % (
        x,
        y
    )
    scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)

## webpage search functions

Here we have a look at all webpages which were used for automatic download. One webpage = one function. E. g.  bitmidicom() downloads from bitmidi.com .

In [40]:
def bitmidicom(search_term:str, song_name:str, download_path_mp3code:str):
    ''' DOWNLOAD, also other versions downloaded'''
    
    global day_limit_counter
    # the target we want to open  
    search_term = search_term.replace(' ', '-').replace('&','and').replace("'","").replace('!','').replace('?','')
    song_name = song_name.replace(' ', '-').replace('&','and').replace("'","").replace('!','').replace('?','')
    url='https://bitmidi.com/' + search_term   + '-mid'
    
    # open with GET method:
    resp=requests.get(url, headers={'User-Agent': 'anything'})

    if resp.status_code==200: # webpage can be opened; artist with song exsting
        
        soup=BeautifulSoup(resp.text,'html.parser') 
        for a in soup.find_all('a'):

            try:
                if 'upload' in a['href']:
                    
                    download_url = 'https://bitmidi.com' + a['href']
                    
                    if day_limit_counter < 500:
                        url_check = webpage_virus_check(download_url)
                        
                        if url_check > -1: # url could be suspicious but don't have to be
                            path_file_updated = download(download_url, download_path_mp3code, 'bitmidicom')
                            
                            if 'text' in magic.from_file(download_path_mp3code): # can be HTML code or text file
                                # like 'You already have reached download limit.' or it is a copy of an existing file
                                os.remove(download_path_mp3code) # delete the unneeded file
                            
                            elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                file_virus_check(path_file_updated)
                                
                            elif url_check == 0 and day_limit_counter >= 500:
                                with open('where_we_stopped_collecting', 'a') as f:
                                    f.write(f'{search_term} bitmidicom \n unfinished file virus check \n {path_file_updated}')
                                return -1
                        else:
                            with open('found_threat_info_undownloaded', 'a') as f:
                                f.write(f'bitmidicom {search_term} {download_url}\n')
                                
                    else:
                        with open('where_we_stopped_collecting', 'a') as f:
                            f.write(f'{search_term} bitmidicom\n unfinished download \n {download_url}')
                        return -1 
                                
            except:
                pass
             

        # finding all other song versions and also download them:
        for s in soup.find_all('div', {'class':'mv4'}):
            if 'Related MIDI Files' in str(s): 
                for ss in s.find_all('a', {'class':'pointer no-underline fw4 white underline-hover'}):
                    # loop over all related midi files for the case we have a second version of
                    # the same song there. Download all versions of one song.

                    try:
                        if song_name in ss['href']:
                            url_next_version = 'https://bitmidi.com' + ss['href']
                            resp_next=requests.get(url_next_version, headers={'User-Agent': 'anything'})
                            soup_next=BeautifulSoup(resp_next.text,'html.parser') 

                            for a_next in soup_next.find_all('a'):

                                try:
                                    if 'upload' in a_next['href']:
                                        download_url_next = 'https://bitmidi.com' + a_next['href']
                                                                              
                                        
                                        if day_limit_counter < 500:
                                            url_check = webpage_virus_check(download_url_next)

                                            if url_check > -1: # url could be suspicious but don't have to be
                                                path_file_updated = download(download_url_next, download_path_mp3code, 'bitmidicom')
                                                
                                                if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                                    # like 'You already have reached download limit.' or it is a copy of an existing file
                                                    os.remove(path_file_updated) # delete the unneeded file

                                                elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                                    file_virus_check(path_file_updated)

                                                elif url_check == 0 and day_limit_counter >= 500:
                                                    with open('where_we_stopped_collecting', 'a') as f:
                                                        f.write(f'{search_term} bitmidicom \n unfinished file virus check \n {path_file_updated}')
                                                    return -1
                                            else:
                                                with open('found_threat_info_undownloaded', 'a') as f:
                                                    f.write(f'bitmidicom {search_term} {download_url}\n')
                                        else:
                                            with open('where_we_stopped_collecting', 'a') as f:
                                                f.write(f'{search_term} bitmidicom\n unfinished download \n {download_url}')
                                            return -1
                        
                                        break
                                except:
                                    pass
                    except:
                        pass
                break
        
        return 1

    else:
        # song does not exist on bitmidi.com
        # answer()
        return 0 
    
# Testing the download success with one example song for which we know that it already exists for this
# webpage. Test download folder 'tester' has to be created:
#os.makedirs('tester', exist_ok=True)
#bitmidicom(search_term = 'lady gaga just dance', song_name='just dance', download_path_mp3code='tester/bitmidicom_idmp.mid')

1

In [41]:
def freemidiorg(searched_artist:str, searched_song:str,download_path_mp3code:str):
    '''DOWNLOAD WITH CLICK BECAUSE SOMETIMES ONLY THROUGH CLICKING DOWNLOAD POSSIBLE'''
    
    global day_limit_counter
    
    full_path = os.path.abspath(download_path_mp3code)
    wished_file_name = full_path.split('/')[-1]
    download_path = full_path.replace(wished_file_name, '')

    # needed for selenium:
    option = webdriver.FirefoxOptions()
    option.headless = True # GUI does not open, but navigation through web suceeds
    # determine where to save the downloaded files
    #0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    option.set_preference("browser.download.folderList", 2) 
    option.set_preference("browser.download.dir", download_path)

    searched_artist = searched_artist.replace('!','').replace('?','').lower()
    searched_song = searched_song.lower().replace('&','and').replace("'","").replace('!','').replace('?','')

    artist_start_letter = searched_artist[0]
    artist_page_url = 'https://freemidi.org/artists-' + artist_start_letter
    resp = requests.get(artist_page_url, headers={'User-Agent':'anything'})
    if resp.status_code==200: # webpage can be opened

        soup = BeautifulSoup(resp.text, 'html.parser')
        for a in soup.find_all('a'):

            if searched_artist == a.get_text().lower():
                url_songs_artist = 'https://freemidi.org/' + a['href']
                resp2 = requests.get(url_songs_artist, headers={'User-Agent':'anything'})
                soup2 = BeautifulSoup(resp2.text, 'html.parser')

                for a2 in soup2.find_all('a'): # loop over all possible versions of one song
                    if searched_song in a2.get_text().lower():
                        url_song = 'https://freemidi.org/' + a2['href']
                        driver = webdriver.Firefox(options=option)
                        driver.get(url_song)

                        ele = driver.find_element(by=By.LINK_TEXT, value='Download MIDI')

                        if day_limit_counter < 500:
                            url_check = webpage_virus_check(url_song)

                            if url_check > -1: # url could be suspicious but don't have to be
                                ele.click() # download by click
                                time.sleep(1) # wait a second before renaming file. else we get an error 
                                path_file_updated = rename_downloaded_file(download_path_mp3code, 'freemidiorg')                               
                               
                                if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                    # like 'You already have reached download limit.' or it is a copy of an existing file
                                    os.remove(path_file_updated) # delete the unneeded file
                                
                                elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                    file_virus_check(path_file_updated)

                                elif url_check == 0 and day_limit_counter >= 500:
                                    with open('where_we_stopped_collecting', 'a') as f:
                                        f.write(f'{searched_artist} {searched_song} freemidiorg \n unfinished file virus check \n {path_file_updated}')
                                    return -1
                            else:
                                with open('found_threat_info_undownloaded', 'a') as f:
                                    f.write(f'freemidiorg {searched_artist} {searched_song}  {ele}\n')
                        else:
                            with open('where_we_stopped_collecting', 'a') as f:
                                f.write(f'{searched_artist} {searched_song} freemidiorg\n unfinished download \n {ele}')
                            return -1                        
                        
                break
    else:
        answer()

#freemidiorg(searched_artist = 'lady gaga', searched_song = 'poker face', download_path_mp3code='tester/freemidiorg_P_LadyGaga_Pokerfa.mid')

/home/c/Schreibtisch/ba_08102022/stage1_data_collecting_phase/webscraping/tester/


In [43]:
def midiworldcom(searched_artist:str, searched_song:str, download_path_mp3code:str):
    ''' DOWNLOAD
    type: following the architecture of the webpage by using url and searching the 
    result lists. In the urls are not the song titles and artist names but only numbers,
    therefore, use soup.find_all() to get overview over page content and finding the
    correct url links. '''
    
    global day_limit_counter
    
    # the target we want to open  
    searched_artist = searched_artist.lower()
    starting_letter = (searched_artist[0]).upper()
    
    searched_song = searched_song.lower().replace('&','and').replace("'","").replace('!','').replace('?','')
    
    for i in ['', range(100)]: # url changes with 
        
        url1 = f"https://www.midiworld.com/files/{starting_letter}/all/" + str(i)#https://www.partnersinrhyme.com/midi/midiartists/index.shtml"
        soup = BeautifulSoup(requests.get(url1).text)
        
        if soup.find('table',{'border':'0'}) is None: # we find no results 
            break
        
        resp=requests.get(url1, headers={'User-Agent': 'anything'})

        if resp.status_code==200:

            for a in soup.find_all('a'): # 'tr',{'valign':'top'}): # get the whole content of the webpage: download pages or subpages containing 

                # 'a' looks like: '<a href="https://www.midiworld.com/files/123/">The Association</a>'
                a_formulation_str = str(a).lower()

                if searched_artist in a_formulation_str:

                    url_artist = str(a['href'])
                    soup_artist = BeautifulSoup(requests.get(url_artist).text)

                    for li in soup_artist.find_all('li') : # in them the song titles are saved

                        if searched_song in str(li).lower(): 
                            for download_info in li.find_all('a'):
                                download_url = download_info['href']
                                
                                
                                
                                if day_limit_counter < 500:
                                    url_check = webpage_virus_check(download_url)

                                    if url_check > -1: # url could be suspicious but don't have to be
                                        path_file_updated = download(download_url, download_path_mp3code, 'midiworldcom')

                                        if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                            # like 'You already have reached download limit.' or it is a copy of an existing file
                                            os.remove(path_file_updated) # delete the unneeded file
                                        
                                        elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                            file_virus_check(path_file_updated)

                                        elif url_check == 0 and day_limit_counter >= 500:
                                            with open('where_we_stopped_collecting', 'a') as f:
                                                f.write(f'{search_term} midiworldcom \n unfinished file virus check \n {path_file_updated}')
                                            return -1
                                    else:
                                        with open('found_threat_info_undownloaded', 'a') as f:
                                            f.write(f'midiworldcom {search_term} {download_url}\n')
                                else:
                                    with open('where_we_stopped_collecting', 'a') as f:
                                        f.write(f'{search_term} midiworldcom\n unfinished download \n {download_url}')
                                    return -1                                
                                
                                # downloaded
                            break

        else:
            answer()

    return

#midiworldcom(searched_artist = 'The Association', searched_song = 'Along Comes Mary',download_path_mp3code='tester/association_idmp.mid')

In [44]:
midisfreecom_download_counter = 10
def midisfreecom(search_term:str, download_path_mp3code:str):
    '''type: searches for single midi files that are returned by webpage'''
    
    # the target we want to open  
    search_term = search_term.replace(' ', '+').replace('&','and').replace("'","").replace('.','+').replace('!','').replace('?','')
    url='https://midisfree.com/search/?search=' + search_term# single_search #'http://www.hindustantimes.com/top-news'
    global day_limit_counter, midisfreecom_download_counter
    midisfreecom_download_counter = 10 # every time midisfreecom() executed change vpn location 
    # in lines below;
    # newly added code
    
    full_path = os.path.abspath(download_path_mp3code)
    wished_file_name = full_path.split('/')[-1]
    download_path = full_path.replace(wished_file_name, '')

    # needed for selenium:
    option = webdriver.FirefoxOptions()
    option.headless = True # GUI does not open, but navigation through web suceeds
    # determine where to save the downloaded files
    #0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    option.set_preference("browser.download.folderList", 2) 
    option.set_preference("browser.download.dir", download_path)
    
    
    driver = webdriver.Firefox(options=option)
    driver.get(url)

    links_to_song_versions = driver.find_element(By.CLASS_NAME, "row").find_elements(By.TAG_NAME, 'a')
    
    collector = [i.get_attribute('href') for i in links_to_song_versions]
    collector = set(collector)

    for url_page_song in collector:
        
        # driver.get(url_page_song)

        if midisfreecom_download_counter >= 4: # download limit per day: 4 files
            os.system("windscribe connect") # change IP
            # driver.quit()
            driver = webdriver.Firefox(options=option)
            # driver.refresh()
            midisfreecom_download_counter = 0

        driver.get(url_page_song)

        # midi = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "download"))) # wait exactly 20 seconds #This is a dummy element
        wait = WebDriverWait(driver, 30, poll_frequency=1)#, ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException])
        midi = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@id='download']"))) # wait at most 30 seconds
        
        if day_limit_counter < 500:
            url_check = webpage_virus_check(url_page_song) 
            print('url_check done')
            if url_check > -1: # url could be suspicious but don't have to be
                try:
                    midi.click()
                    print(midi)
                except:
                    if 'firefox' in driver.capabilities['browserName']:
                        scroll_shim(driver, midi )
                    # scroll_shim is just scrolling it into view, you still need to hover over it to click using an action chain.
                    actions = ActionChains(driver)
                    actions.move_to_element(midi )
                    actions.click()
                    actions.perform() # downloading                               
                    time.sleep(1)
                    
                    
                print('downloaded path with name',download_path_mp3code)
                midisfreecom_download_counter += 1
           
                
                path_file_updated = rename_downloaded_file(download_path_mp3code, 'midisfreecom')

                if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                    # like 'You already have reached download limit.' or it is a copy of an existing file
                    os.remove(path_file_updated) # delete the unneeded file
 
                elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                    file_virus_check(path_file_updated)
                elif url_check == 0 and day_limit_counter >= 500:
                    with open('where_we_stopped_collecting', 'a') as f:
                        f.write(f'{search_term} midisfreecom \n unfinished file virus check \n {path_file_updated}')
                    return -1
            else:
                with open('found_threat_info_undownloaded', 'a') as f:
                    f.write(f'midisfreecom {search_term} {url_page_song}\n')
        else:
            with open('where_we_stopped_collecting', 'a') as f:
                f.write(f'{search_term} midisfreecom\n unfinished download \n {url_page_song}')
            return -1
        
    return
    
#midisfreecom(search_term = 'shakira hips dont lie',download_path_mp3code='tester/hips_idmp.mid')
#midisfreecom(search_term = 'nirvana blew',download_path_mp3code='tester/freemidiUMBAU_idmp.mid')

Connecting to Germany Frankfurt Wurstchen (UDP:443)
Firewall Enabled
Connected to Germany Frankfurt Wurstchen
Your IP changed from 45.87.212.23 to 149.36.50.7


TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


In [45]:
def cpratocom(search_term:str, download_path_mp3code:str):
    '''DOWNLOAD
    search_term: artist song 
    type: searches for single midi files that are returned by webpage. 
    When webpage does not find result it returns "No MIDI files found." '''
    
    global day_limit_counter
    
    # the target we want to open  
    search_term = search_term.replace(' ', '-').replace('&','and').replace("'","").replace('!','').replace('?','')
    url='https://www.cprato.com/en/midi/search/' + search_term# single_search #'http://www.hindustantimes.com/top-news'
      
    #open with GET method
    resp=requests.get(url, headers={'User-Agent': 'anything'})

    if resp.status_code==200: # webpage can be opened

        soup=BeautifulSoup(resp.text,'html.parser')   
        
        for a in soup.find_all('a'):
            
            if 'Free Download' in str(a): # we find the download link
                download_url1 = a['href']
                download_url = 'https://www.cprato.com' + download_url1                
            
                full_path = os.path.abspath(download_path_mp3code)
                wished_file_name = full_path.split('/')[-1]
                download_path = full_path.replace(wished_file_name, '')
                
                if day_limit_counter < 500:
                    
                    # needed for selenium:
                    option = webdriver.FirefoxOptions()
                    option.headless = True # GUI does not open, but navigation through web suceeds
                    # determine where to save the downloaded files
                    #0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
                    option.set_preference("browser.download.folderList", 2) 
                    option.set_preference("browser.download.dir", download_path)

                    
                    url_check = webpage_virus_check(download_url)

                    if url_check > -1: # url could be suspicious but don't have to be
                        
                        driver = webdriver.Firefox(options=option)
                        driver.get(download_url)
                        time.sleep(6)
                        path_file_updated = rename_downloaded_file(download_path_mp3code, 'cpratocom')

                        if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                            # like 'You already have reached download limit.' or it is a copy of an existing file
                            os.remove(path_file_updated) # delete the unneeded file
                        
                        elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                            file_virus_check(path_file_updated)

                        elif url_check == 0 and day_limit_counter >= 500:
                            with open('where_we_stopped_collecting', 'a') as f:
                                f.write(f'{search_term} cpratocom \n unfinished file virus check \n {path_file_updated}')
                            return -1
                    else:
                        with open('found_threat_info_undownloaded', 'a') as f:
                            f.write(f'cpratocom {search_term} {download_url}\n')
                else:
                    with open('where_we_stopped_collecting', 'a') as f:
                        f.write(f'{search_term} cpratocom\n unfinished download \n {download_url}')
                    return -1
                
                break
        
    else:
        answer()
    return

#cpratocom(search_term = 'alan walker faded',download_path_mp3code='tester/walkerFaded_idmp.mid')

/home/c/Schreibtisch/ba_08102022/stage1_data_collecting_phase/webscraping/tester/


In [35]:
#copy old
nonstop2kcom_download_counter = 2 # counts how often we downloaded from nonstop2k.com

def nonstop2kcom(search_term:str,download_path_mp3code:str):
    '''DOWNLOAD, 
    
    search_term: artist song 
    type: searches for single midi files that are returned by webpage. When webpage does not find result
    it returns "No MIDI files found.".
    
    ARGUMENT download_path_mp3code: Be aware that there have to be a '?' in the path,
    e. g. MP3codeFolderName/?/FileName.mid,
    The '?' defines if we have a full or demo version. This function finds it out.
    
    We only open the webpage when we found the a song for FREE. 
    This is reached with the end part of the url.'''

    global day_limit_counter, nonstop2kcom_download_counter
    # nonstop2kcom_download_counter = 2 # do this when problems with limited downloading
    
    # the target we want to open  
    search_term1 = search_term.replace(' ', '+').replace('&','and').replace("'","").replace('!','').replace('?','')

    url='https://www.nonstop2k.com/midi-files/search.php?keywords=' + search_term1 + '&send=1&type=all&kid=0&d=0&pm=2&f=0&act=3' # to get only the free songs# single_search #'http://www.hindustantimes.com/top-news'

    global nonstop2kcom_download_counter 
    #open with GET method
    resp=requests.get(url, headers={'User-Agent': 'anything'})

    if resp.status_code==200: # webpage can be opened
        
        soup=BeautifulSoup(resp.text,'html.parser')    
      
        # results is the list which contains all the text:
        page_content = soup.find('div', {'class':'heroContainer'}) # returns None when we found a result
        
        if page_content is None: # we found a result
            search_term2 = search_term.replace(' ', '-')
            search_term2 = search_term2.lower()
            for a in soup.find_all('a'):

                try:
                    if search_term2 in a['href'].lower() and 'midi.html' in a['href'].lower():

                        url2 = 'https://www.nonstop2k.com' + a['href'][2:]
                        # url2 gives us the webpage with the song title and 
                        # download link
                                               
                        resp2=requests.get(url2, headers={'User-Agent': 'anything'})
                        soup2=BeautifulSoup(resp2.text,'html.parser') 
                        
                        
                        # find out if we have a demo or full version available:
                        for i in soup2.find('div', {'id':'midifeatures'}).find_all('span'):
                            try:
                                if "midiNotFull" in i['class']: # then we have a demo of the song
                                    download_path_mp3code = download_path_mp3code.replace('?', 'demo')
                                    break
                                elif "midiFull" in i['class']: # then we get the full song
                                    download_path_mp3code = download_path_mp3code.replace('?', 'full')
                                    break
                            except:
                                pass

                        
                        # download the found midi file:
                      
                        if day_limit_counter < 500:

                            full_path = os.path.abspath(download_path_mp3code)
                            wished_file_name = full_path.split('/')[-1]
                            download_path = full_path.replace(wished_file_name, '')

                            # Overgo download limit of 2 files per day:
                            if nonstop2kcom_download_counter == 2:
                                os.system("windscribe connect")
                                nonstop2kcom_download_counter = 0

                            # needed for selenium:

                            option = webdriver.FirefoxOptions()
                            option.headless = True # GUI does not open, but navigation through web suceeds
                            # determine where to save the downloaded files
                            #0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
                            option.set_preference("browser.download.folderList", 2) 
                            option.set_preference("browser.download.dir", download_path)

                            
                            driver = webdriver.Firefox(options=option)
                            driver.get(url2)

                            download_ele = driver.find_element(By.ID, 'midiDlB1')
                                                     
                            url_check = webpage_virus_check(url2) 
                           
                            if url_check > -1: # url could be suspicious but don't have to be

                                if 'firefox' in driver.capabilities['browserName']:
                                    scroll_shim(driver, download_ele )
                                # scroll_shim is just scrolling it into view, you still need to hover over it to click using an action chain.
                                actions = ActionChains(driver)
                                actions.move_to_element(download_ele )
                                actions.click()
                                actions.perform() # downloading                               
                                nonstop2kcom_download_counter += 1
                                
                                time.sleep(1) # renaming works after waiting 1 second
                                path_file_updated = rename_downloaded_file(download_path_mp3code, 'nonstop2kcom') 

                                if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                    # like 'You already have reached download limit.' or it is a copy of an existing file
                                    os.remove(path_file_updated) # delete the unneeded file
                                
                                elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                    file_virus_check(path_file_updated)

                                elif url_check == 0 and day_limit_counter >= 500:
                                    with open('where_we_stopped_collecting', 'a') as f:
                                        f.write(f'{search_term} nonstop2kcom \n unfinished file virus check \n {path_file_updated}')
                                    return -1
                            else:
                                with open('found_threat_info_undownloaded', 'a') as f:
                                    f.write(f'nonstop2kcom {search_term} {url2}\n')
                        else:
                            with open('where_we_stopped_collecting', 'a') as f:
                                f.write(f'{search_term} nonstop2kcom\n unfinished download \n {url2}')
                            return -1
                        
                        return

                except:
                    pass
        
    else:
        answer()
        
    return

# nonstop2kcom(search_term = 'lady gaga hair',download_path_mp3code='tester/?/nonstop_idmp.mid')
# nonstop2kcom(search_term = 'queen bohemian rhapsody',download_path_mp3code='tester/?/nonstop_idmp.mid')

In [22]:
def partnersinrhymecom(searched_artist:str, searched_song:str, download_path_mp3code):
    ''' DOWNLOAD!
    searched_artist: type in artist name
    type: want to find artist via url. when page gives back 404, we know that artist not exists for this
    webpage. Same for song.
    '''
    
    global day_limit_counter
    
    # the target we want to open  
    searched_artist = searched_artist.lower()
    searched_artist = searched_artist.replace(' ', '_')
    
    searched_song = searched_song.replace(' ', '_').replace('&','and').replace("'","").replace('!','').replace('?','')
    searched_song = searched_song.lower()
        
    download_url = f'https://composerconnection.com/midi1/pop_rock/{searched_artist}/{searched_song}.mid'
    # url found out with soup.find_all() function
    
    resp=requests.get(download_url, headers={'User-Agent': 'anything'})
        
    if resp.status_code==200: # webpage can be opened
        # response 404 when webpage with given artist name and song does not exist!
        
        if day_limit_counter < 500:
            url_check = webpage_virus_check(download_url)

            if url_check > -1: # url could be suspicious but don't have to be
                path_file_updated = download(download_url, download_path_mp3code, 'partnersinrhymecom')
                
                if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                    # like 'You already have reached download limit.' or it is a copy of an existing file
                    os.remove(path_file_updated) # delete the unneeded file

                elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                    file_virus_check(path_file_updated)

                elif url_check == 0 and day_limit_counter >= 500:
                    with open('where_we_stopped_collecting', 'a') as f:
                        f.write(f'{search_term} partnersinrhymecom \n unfinished file virus check \n {path_file_updated}')
                    return -1
            else:
                with open('found_threat_info_undownloaded', 'a') as f:
                    f.write(f'partnersinrhymecom{search_term} {download_url}\n')
        else:
            with open('where_we_stopped_collecting', 'a') as f:
                f.write(f'{search_term} partnersinrhymecom\n unfinished download \n {download_url}')
            return -1
       
    return

# partnersinrhymecom(searched_artist = 'nirvana',searched_song='blew',download_path_mp3code='tester/BLEW_idmp.mid')

In [23]:
def mididbcom(searched_artist:str, searched_song:str, download_path_mp3code:str):
    '''
    searched_artist: type in artist name
    type: want to find artist + song via url. when page gives back 404, we know that artist + song
    not exists for this webpage
    '''
    
    global day_limit_counter
    
    # the target we want to open  
    searched_artist = searched_artist.replace(' ', '-').lower()
    searched_song = searched_song.replace(' ', '-').replace('&','and').replace("'","").replace('!','').replace('?','').lower()
    
    
    url1 = 'https://www.mididb.com/'
    url2 = '-midi/'# single_search #'http://www.hindustantimes.com/top-news'
    url = url1 + searched_artist + '/'+ searched_song + url2 

    #open with GET method:
    resp=requests.get(url, headers={'User-Agent': 'anything'})
        
    if resp.status_code==200: # webpage can be opened
        # response 404 when webpage with given artist + song does not exist!

        soup = BeautifulSoup(requests.get(url).text)
        for a in soup.find_all('a'):
            try:
                if 'download' in a['href']:
                    
                    download_url = a['href']
                    
                    if day_limit_counter < 500:
                        url_check = webpage_virus_check(download_url)

                        if url_check > -1: # url could be suspicious but don't have to be
                            path_file_updated = download(download_url, download_path_mp3code, 'mididbcom')

                            if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                # like 'You already have reached download limit.' or it is a copy of an existing file
                                os.remove(path_file_updated) # delete the unneeded file
                            
                            elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                file_virus_check(path_file_updated)

                            elif url_check == 0 and day_limit_counter >= 500:
                                with open('where_we_stopped_collecting', 'a') as f:
                                    f.write(f'{search_term} mididbcom \n unfinished file virus check \n {path_file_updated}')
                                return -1
                        else:
                            with open('found_threat_info_undownloaded', 'a') as f:
                                f.write(f'mididbcom {search_term} {download_url}\n')
                    else:
                        with open('where_we_stopped_collecting', 'a') as f:
                            f.write(f'{search_term} mididbcom\n unfinished download \n {download_url}')
                        return -1                   
                    
                    break
            except:
                pass
                              
    return

# mididbcom(searched_artist = 'nirvana',searched_song='come as you are',download_path_mp3code='tester/mididbcom_idmp.mid')

In [24]:
miditune_download_counter = 43
def miditunecom(searched_artist:str, searched_song:str, download_path_mp3code:str): 
    ''' DOWNLOAD
    Has a download limit of 42 files per day. Therefore, we simply change the IP address when exceeding
    this limit.
    type: the url does not change here when we search for a song on the webpage. Therefore, we have a look at
    all download files of this webpage!'''
    
    global day_limit_counter
    
    global miditune_download_counter
    # miditune_download_counter = 43 # activate this line when problems with limited daily download amount
    
    full_path = os.path.abspath(download_path_mp3code)
    wished_file_name = full_path.split('/')[-1]
    download_path = full_path.replace(wished_file_name, '')
    
    # needed for selenium:
    option = webdriver.FirefoxOptions()
    option.headless = True # GUI does not open, but navigation through web suceeds
    # determine where to save the downloaded files
    #0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    option.set_preference("browser.download.folderList", 2) 
    option.set_preference("browser.download.dir", download_path)
    
    searched_song = searched_song.replace('&','and').replace("'","").replace('!','').replace('?','').lower()
    searched_artist = searched_artist.lower()
    searched_string = searched_artist + ' - ' + searched_song
                
    searched_song_strich = searched_song.replace(' ', '-')
    searched_artist_strich = searched_artist.replace(' ', '-')
    url_alphabet_list = f'http://miditune.com/list/midi-{searched_artist[0].upper()}.html' # found out via soup
    resp=requests.get(url_alphabet_list, headers={'User-Agent': 'anything'})
    if resp.status_code==200:
        
        soup = BeautifulSoup(requests.get(url_alphabet_list).text)
        
        # Loop over all artist-song-combis to get all versions of one song:
        for aa in soup.find_all('a'):

            if searched_song in str(aa).lower() and searched_artist in str(aa).lower():
                    try:
                        next_url_part = aa['href'] # page where we will find 
                        important = next_url_part.split('/')[3]
                        next_url = 'http://download.midimi.org/' + important
                    
                        driver = webdriver.Firefox(options=option)
                        driver.get(next_url)
                        find_link_to_download = driver.find_elements(By.TAG_NAME, "a")
                     
                        # Loop over all possible 'a' html parts to find the download link of our song:
                        for e in find_link_to_download:
                            
                            searched_string2 = searched_string + '.mid'
                           
                        
                            if '#' in str(e.get_attribute("href")) and searched_song in str(e.text).lower() and searched_artist in str(e.text).lower() and '.mid' in str(e.text).lower():
                                download_url =  e.get_attribute("href")

                                if miditune_download_counter > 42: # download limit of miditune.com
                                    # is by 42 files per day.
                                    # Therefore, simply change IP address to get access to even
                                    # more files.
                                    driver.close()
                                    os.system("windscribe connect") # change IP
                                   
                                    driver = webdriver.Firefox(options=option)
                                    driver.get(next_url)
                                    
                                    find_link_to_download = driver.find_elements(By.TAG_NAME, "a")
                                    for e2 in find_link_to_download:
                                  
                                        if '#' in str(e2.get_attribute("href")) and searched_song in str(e2.text).lower() and searched_artist in str(e2.text).lower() and '.mid' in str(e2.text).lower():
                                            
                                            if day_limit_counter < 500:
                                                url_check = webpage_virus_check(download_url)
                                                
                                                if url_check > -1: # url could be suspicious but don't have to be
                                                    e2.click() # click on download link
                                                   
                                                    path_file_updated =  rename_downloaded_file(download_path_mp3code, 'miditunecom')
                                                    
                                                    if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                                        # like 'You already have reached download limit.' or it is a copy of an existing file
                                                        os.remove(path_file_updated) # delete the unneeded file
                                                    
                                                    elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                                        file_virus_check(path_file_updated)

                                                    elif url_check == 0 and day_limit_counter >= 500:
                                                        with open('where_we_stopped_collecting', 'a') as f:
                                                            f.write(f'{search_term} miditunecom \n unfinished file virus check \n {path_file_updated}')
                                                        return -1
                                                else:
                                                    with open('found_threat_info_undownloaded', 'a') as f:
                                                        f.write(f'miditunecom {search_term} {next_url}\n')
                                            else:
                                                with open('where_we_stopped_collecting', 'a') as f:
                                                    f.write(f'{search_term} miditunecom\n unfinished download \n {next_url}')
                                                return -1

                                            miditune_download_counter = 1
                                            break
                                else:
                                    
                                    if day_limit_counter < 500:
                                        
                                        url_check = webpage_virus_check(download_url)
                                       
                                        if url_check > -1: # url could be suspicious but don't have to be
                                            e.click() # click on download link
                                            
                                            path_file_updated =  rename_downloaded_file(download_path_mp3code, 'miditunecom')
                                          
                                            if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                                # like 'You already have reached download limit.' or it is a copy of an existing file
                                                os.remove(path_file_updated) # delete the unneeded file
       
                                            elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                                file_virus_check(path_file_updated)

                                            elif url_check == 0 and day_limit_counter >= 500:
                                                with open('where_we_stopped_collecting', 'a') as f:
                                                    f.write(f'{search_term} miditunecom \n unfinished file virus check \n {path_file_updated}')
                                                return -1
                                        else:
                                            with open('found_threat_info_undownloaded', 'a') as f:
                                                f.write(f'miditunecom {search_term} {next_url}\n')
                                    else:
                                        with open('where_we_stopped_collecting', 'a') as f:
                                            f.write(f'{search_term} miditunecom\n unfinished download \n {next_url}')
                                        return -1 
                                                                        
                                    miditune_download_counter += 1
                                    driver.close()
                                    # downloaded
                                break                                 

                    except:
                        pass
      
    else:
        answer()

    return                                    
                                    
# miditunecom(searched_artist='nirvana', searched_song='come as you are', download_path_mp3code='tester/miditune_wishname.mid')#download_path='/home/c/Schreibtisch/ba/yo')

In [25]:
def lvbeethovenfr(searched_artist:str):
    '''
    NO download, only URL opening.
    
    This webpage only contains Beethoven.
    type: only look if the searched artist is Beethoven. When that is the case,
    open the webpage

    '''
    
    searched_artist = searched_artist.lower()
    if searched_artist == 'beethoven':

       # url = "http://www.lvbeethoven.fr/Oeuvres/Music-Midi-Mp3-Symphonies.html"
        url = 'http://www.lvbeethoven.fr/Midi/index_En.html'
        resp=requests.get(url, headers={'User-Agent': 'anything'})

        if resp.status_code==200:
            
            open_(url)
           
        else:
            answer()

#lvbeethovenfr(searched_artist='beethoven')

In [26]:
def rppmfcom(searched_artist:str, searched_song:str, download_path_mp3code:str):
    ''' DIRECT DOWNLOAD
    searched_artist: type in artist name
    type: want to find artist + song via url. when page gives back 404, we know that artist + song
    not exists for this webpage
    '''
    
    global day_limit_counter
    
    # the target we want to open  
    searched_artist = searched_artist.replace(' ', '_')
    searched_artist = searched_artist.lower()
    searched_song = searched_song.replace(' ', '_').replace('&','and').replace("'","").replace('!','').replace('?','')
    searched_song = searched_song.lower()
    
    url1 = 'http://www.rppmf.com/'
    url2 = '.htm'# single_search #'http://www.hindustantimes.com/top-news'
    url = url1 + searched_artist + url2 
    
    #open with GET method
    
    resp=requests.get(url, headers={'User-Agent': 'anything'})

    if resp.status_code==200: # webpage can be opened # artist exists
        # response 404 when webpage with given artist does not exist!

        soup = BeautifulSoup(requests.get(url).text)

        for a in soup.find_all('a'): # get the whole content of the webpage: download pages or subpages containing 
        # further downloads
       
            try:
                midi_path = a['href']
                if '.mid' in midi_path and searched_song in midi_path: # download the midi file!
                    download_url = url1 + midi_path                  
                    
                    if day_limit_counter < 500:
                        url_check = webpage_virus_check(download_url)

                        if url_check > -1: # url could be suspicious but don't have to be
                            path_file_updated = download(download_url, download_path_mp3code, 'rppmfcom')
                            
                            if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                # like 'You already have reached download limit.' or it is a copy of an existing file
                                os.remove(path_file_updated) # delete the unneeded file
                            
                            elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                file_virus_check(path_file_updated)

                            elif url_check == 0 and day_limit_counter >= 500:
                                with open('where_we_stopped_collecting', 'a') as f:
                                    f.write(f'{search_term} rppmfcom \n unfinished file virus check \n {path_file_updated}')
                                return -1
                        else:
                            with open('found_threat_info_undownloaded', 'a') as f:
                                f.write(f'rppmfcom {search_term} {download_url}\n')
                    else:
                        with open('where_we_stopped_collecting', 'a') as f:
                            f.write(f'{search_term} rppmfcom\n unfinished download \n {download_url}')
                        return -1
                    
            except:
                pass
    
    return

#rppmfcom(searched_artist = 'coldplay',searched_song='spies',download_path_mp3code='tester/spies_idmp.mid')

In [27]:
def metalmidigrahamdowneycom(searched_artist:str, searched_song:str, download_path_mp3code:str):
    ''' DIRECT DOWNLOAD
    searched_artist: type in artist name
    type: want to find artist + song via url. when page gives back 404, we know that artist + song
    not exists for this webpage
    '''
    
    global day_limit_counter
    
    # the target we want to open  

    searched_artist = searched_artist.lower()
  
    searched_song = searched_song.replace('&','and').replace("'","").replace('!','').replace('?','').lower()
    
    url1 = 'http://metal-midi.grahamdowney.com/'
    url2 = 'midi.html'# single_search #'http://www.hindustantimes.com/top-news'
    url = url1 + url2  # general urls
    
    #open with GET method:
    resp=requests.get(url, headers={'User-Agent': 'anything'}) 
        
    if resp.status_code==200: # webpage can be opened # artist exists
        # response 404 when webpage with given artist does not exist!

        soup = BeautifulSoup(requests.get(url).text)

        for a in soup.find_all('a'): # get the whole content of the webpage: download pages or subpages containing 
        # further downloads
       
            try:
                midi_path = a['href']
                midi_path_name = midi_path.lower()
                if '.mid' in midi_path_name and searched_artist in midi_path_name and searched_song in midi_path_name: # download the midi file!
                    download_url = url1 + midi_path
           
                    if day_limit_counter < 500:
                        url_check = webpage_virus_check(download_url)

                        if url_check > -1: # url could be suspicious but don't have to be
                            path_file_updated = download(download_url, download_path_mp3code, 'metalmidigrahamdowneycom')
                            
                            if 'text' in magic.from_file(path_file_updated): # can be HTML code or text file
                                # like 'You already have reached download limit.' or it is a copy of an existing file
                                os.remove(path_file_updated) # delete the unneeded file
                            
                            elif url_check == 0 and day_limit_counter < 500: # url is more or less suspicious
                                file_virus_check(path_file_updated)

                            elif url_check == 0 and day_limit_counter >= 500:
                                with open('where_we_stopped_collecting', 'a') as f:
                                    f.write(f'{search_term} metalmidigrahamdowneycom \n unfinished file virus check \n {path_file_updated}')
                                return -1
                        else:
                            with open('found_threat_info_undownloaded', 'a') as f:
                                f.write(f'metalmidigrahamdowneycom {search_term} {download_url}\n')
                    else:
                        with open('where_we_stopped_collecting', 'a') as f:
                            f.write(f'{search_term} metalmidigrahamdowneycom\n unfinished download \n {download_url}')
                        return -1            
            
            except: 
                pass
            
    else:
        answer()

    return

#metalmidigrahamdowneycom(searched_artist = 'chinese man',searched_song='ive got that tune',download_path_mp3code='tester/fail_idmp.mid')
#metalmidigrahamdowneycom(searched_artist = 'Type O Negative',searched_song='Black No.1',download_path_mp3code='tester/blockno1_idmp.mid')

## adding all functions together and go through data set
Go through each sample and download from webpages above with help of webpage functions.

In [ ]:
day_limit_counter = 0

In [39]:
col = small3.columns
number = int(input('Please, type starting index in it:'))
small4 = small3[number:] # start in the dataframe with the index+1 where we stopped downloading last time

for ind, row in small4.iterrows():
    curr_code = row[col[0]]
    curr_mp3 =  row[col[1]]
    curr_art =  row[col[2]]
    curr_song =  row[col[3]]
    curr_search_term =  row[col[4]]
    curr_search_term2 =  row[col[5]]
    
    
    day_limit_counter_old = day_limit_counter      
    
    download_path = 'midi-files/' + str(curr_mp3) + '/full/'
    
    download_path_demo = 'midi-files/' + str(curr_mp3) + '/demo/'
    
    # remove all files downloaded so far for the starting index to avoid duplicates:
    if ind == number:
        shutil.rmtree(download_path) # remove folder
        os.makedirs(download_path) # create same folder but its empty now
        
        shutil.rmtree(download_path_demo)
        os.makedirs(download_path_demo)
        
        path3a = os.path.join(download_path, 'webpageinfo.txt')
        path4a = os.path.join(download_path_demo, 'webpageinfo.txt')
        open(path3a,'w') # create file in which the related web pages shall be stored
        open(path4a,'w')
        
        
    
    # handle beethoven page:
    art_lower = curr_art.lower()
    if art_lower == 'beethoven':
        print(row)
        print(f'folder: {curr_mp3}')
        lvbeethovenfr(art_lower)
        v = input('How many files did you download?')
        day_limit_counter += int(v)
        done = False
        if int(v) == 0:
            done = True
        while not done:
            name_mid = input('what is one of the filenames?[name/done]')
            if name_mid == 'done':
                done = True
            else:
                path_name = 'midi-files/' + curr_mp3 + '/full/' + f'{name_mid}'
                file_virus_check(path_name)
                
        
    # getting remaining pages with automatic download:
    a=bitmidicom(search_term = curr_search_term, song_name=curr_song, download_path_mp3code=(download_path+'bitmidicom_'+curr_search_term+'.mid'))
    print('a done')
    b=freemidiorg(searched_artist = curr_art, searched_song = curr_song, download_path_mp3code=(download_path+'freemidiorg_'+curr_search_term+'.mid'))
    print('b done')
    c=midiworldcom(searched_artist = curr_art, searched_song = curr_song,download_path_mp3code=(download_path+'midiworld_'+curr_search_term+'.mid'))
    print('c done')
    d=midisfreecom(search_term = curr_search_term,download_path_mp3code=(download_path+'midisfreecom_'+curr_mp3+'.mid'))
    print('the midisfreedownloadcounter.com', midisfreecom_download_counter)
    print('d done')
    e=cpratocom(search_term = curr_search_term,download_path_mp3code=(download_path+'cpratocom_'+curr_mp3+'.mid'))
    print('e done')
    f=partnersinrhymecom(searched_artist = curr_art,searched_song=curr_song,download_path_mp3code=(download_path+'partnersinrhymecom_'+curr_search_term+'.mid'))
    print('f done')
    g=mididbcom(searched_artist = curr_art,searched_song=curr_song,download_path_mp3code=(download_path+'mididbcom_'+curr_search_term+'.mid'))   
    print('g done')

    h=miditunecom(searched_artist=curr_art, searched_song=curr_song, download_path_mp3code= (download_path+'miditunecom_'+curr_search_term+'.mid'))   
    print('h done')
    
    # seems to be knocked out:
    # when the webpage metalmidigrahamdowney.com works again you can activate it
    #i=metalmidigrahamdowneycom(searched_artist = curr_art,searched_song=curr_song,download_path_mp3code=(download_path+'metalmidigrahamdowneycom_'+curr_search_term+'.mid'))    
    print('i done')
    non2k_path = download_path.replace('full','?')
    j=nonstop2kcom(search_term = curr_search_term,download_path_mp3code=(non2k_path+'nonstop2kcom_'+curr_search_term+'.mid'))
    print('j done')
    
    k = rppmfcom(searched_artist = curr_art,searched_song=curr_song,download_path_mp3code=(download_path+'rppmfcom_'+curr_search_term+'.mid'))    
    print('k done')
    
    # l = kunstderfugecom(searched_artist=curr_art, searched_song=curr_song, download_path_mp3code=(download_path+'kunstderfugecom_'+curr_search_term+'.mid'))
    # page wasn't standardized enough. So automatic downloading did not work out.
    l = 5
    print('l done')
    
    return_list = [a,b,c,d,e,f,g,h,j,k,l]#,i,j,k,l]
    if -1 in return_list:
        with open('index where stopped','w') as g:
            g.write(f'{ind}, have a look to the other generated files.')
    
    # checking if we found files for search term in internet:
    if day_limit_counter == day_limit_counter_old: # we downloaded nothing for a certain song
        with open('final_missing_titles','a') as f:
            f.write(f'{curr_search_term2} {curr_mp3}\n') # getting file with all songs for which we have to find something
    else:
        with open('found_titles','a') as f:
            f.write(f'{curr_search_term2} {curr_mp3}\n')        
    print(ind, '-th row/song_artist done')

Please, type starting index in it:1
https://bitmidi.com/Desiigner-Panda-mid
a done
b done
200
c done
https://midisfree.com/search/?search=Desiigner+Panda
/home/c/Schreibtisch/ba_08102022/stage1_data_collecting_phase/webscraping/midi-files/H_Desiigner_Panda/full/
the midisfreedownloadcounter.com 10
d done
e done
f done
https://www.mididb.com/desiigner/panda-midi/
<Response [404]>
g done
h done
i done
j done
<Response [404]>
http://www.rppmf.com/desiigner.htm
k done
l done
1 -th row/song_artist done
https://bitmidi.com/NAS-Doo-Rags-mid
a done
b done
200
c done
https://midisfree.com/search/?search=NAS+Doo+Rags
/home/c/Schreibtisch/ba_08102022/stage1_data_collecting_phase/webscraping/midi-files/H_NAS_DooRags/full/
the midisfreedownloadcounter.com 10
d done
e done
f done
https://www.mididb.com/nas/doo-rags-midi/
<Response [404]>
g done
h done
i done
j done
<Response [404]>
http://www.rppmf.com/nas.htm
k done
l done
2 -th row/song_artist done
https://bitmidi.com/DMX-X-Gon-Give-It-To-Ya-mid
a

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


#### Comparing found and unfound titles:


In [70]:
with open('found_titles', 'r') as found_songs:

    content = found_songs.read()
    
    with open('no results for song', 'r') as not_found:

        for line in not_found:
            
            if line.strip() not in content:
                with open('final_missing_titles', 'a') as final:
                    final.write(line)

**Later info from 30.07.22: Finally, have a look at final_missing_titles to open kunstderfuge.com and download all needed classic songs.**
